# Anticancer Peptide (ACP) recognition using the BOW

## All necessary imports

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

RND_SEED = 42  # for reproducibility
np.random.seed(RND_SEED)

## Reading the ACP data

In [61]:
# Loading data
df = pd.read_csv('../../data/acp/train_data.csv')
df = df.sample(frac=1, random_state=RND_SEED)
print(df.shape)

(1378, 2)


In [62]:
df.head(5)

,sequences,label
597,GLFDIVKKIAGHIVSSI,1
700,KLKNFAKGVAQSLLNKASCKLSGQC,1
1222,GLWNSIKIAGKKLFVNVLDKIRCKVAGGCKTSPDVE,0
1145,GAFGNFLKGVAKKAGLKILSIAQCKLFGTC,1
602,INLKAIAALAKKLLG,0


In [63]:
print(f'{"Column":15s} # of unique values')
print(f'{"-"*40:60s}')
for col in df:
    print(f'{col:15s} {len(df[col].unique())}')

Column          # of unique values
----------------------------------------                    
sequences       1378
label           2


In [64]:
# Is the data balanced?
df['label'].value_counts(normalize=True)

1    0.5
0    0.5
Name: label, dtype: float64

## Bag Of Words

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
set_chars = set()
for i in list(df["sequences"]):
    set_chars = set_chars | set(i)
list_chars = list(set_chars)
list_chars.sort()
' '.join(list_chars)

'A C D E F G H I K L M N P Q R S T V W Y'

In [67]:
from collections import Counter

In [69]:
counts = [[Counter(i)[list_chars[j]] for j, c in enumerate(list_chars)] for i in list(df["sequences"])]
counts

[[1, 0, 1, 0, 1, 2, 1, 4, 2, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0],
 [3, 2, 0, 0, 1, 2, 0, 0, 5, 4, 0, 2, 0, 2, 0, 3, 0, 1, 0, 0],
 [2, 2, 2, 1, 1, 4, 0, 3, 6, 3, 0, 2, 1, 0, 1, 2, 1, 4, 1, 0],
 [4, 2, 0, 0, 3, 5, 0, 2, 5, 4, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0],
 [4, 0, 0, 0, 0, 1, 0, 2, 3, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 3, 0, 1, 1, 7, 2, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 0, 0, 3, 1, 3, 0, 0, 1, 0, 0],
 [3, 0, 1, 1, 0, 7, 0, 3, 5, 3, 0, 2, 0, 0, 0, 1, 1, 4, 0, 0],
 [1, 6, 1, 1, 4, 3, 0, 0, 5, 3, 1, 1, 2, 0, 3, 4, 3, 1, 1, 1],
 [4, 0, 0, 0, 1, 1, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 2, 0, 2, 4, 0, 2, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0],
 [0, 6, 0, 1, 0, 5, 0, 1, 0, 1, 0, 2, 3, 0, 1, 2, 4, 2, 1, 0],
 [0, 6, 1, 1, 0, 4, 1, 0, 2, 5, 1, 1, 0, 0, 7, 1, 2, 1, 0, 2],
 [1, 0, 0, 1, 2, 6, 1, 2, 0, 0, 0, 0, 2, 0, 0, 1, 1, 2, 0, 2],
 [3, 2, 0, 0, 3, 1, 0, 4, 4, 2, 1, 0, 2, 0, 0, 1, 1, 0,

In [70]:
df_counts = pd.DataFrame(data=counts, columns=list_chars)
df_counts["id"] = list(range(df.shape[0]))
df_counts

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,id
0,1,0,1,0,1,2,1,4,2,1,...,0,0,0,0,2,0,2,0,0,0
1,3,2,0,0,1,2,0,0,5,4,...,2,0,2,0,3,0,1,0,0,1
2,2,2,2,1,1,4,0,3,6,3,...,2,1,0,1,2,1,4,1,0,2
3,4,2,0,0,3,5,0,2,5,4,...,1,0,1,0,1,1,1,0,0,3
4,4,0,0,0,0,1,0,2,3,4,...,1,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,1,0,1,0,0,6,0,0,0,6,...,2,0,0,0,2,2,3,0,0,1373
1374,6,6,1,0,1,5,1,1,1,2,...,2,0,0,4,4,1,3,0,2,1374
1375,0,2,0,0,0,6,0,2,2,5,...,0,0,0,0,2,0,2,0,0,1375
1376,5,6,0,0,0,4,0,0,2,3,...,6,0,1,0,2,3,3,3,1,1376


## Sequence Classification using Deep Learning

In [71]:
X = df_counts.set_index("id")
X

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
id,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,1,2,1,4,2,1,0,0,0,0,0,2,0,2,0,0
1,3,2,0,0,1,2,0,0,5,4,0,2,0,2,0,3,0,1,0,0
2,2,2,2,1,1,4,0,3,6,3,0,2,1,0,1,2,1,4,1,0
3,4,2,0,0,3,5,0,2,5,4,0,1,0,1,0,1,1,1,0,0
4,4,0,0,0,0,1,0,2,3,4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,1,0,1,0,0,6,0,0,0,6,0,2,0,0,0,2,2,3,0,0
1374,6,6,1,0,1,5,1,1,1,2,0,2,0,0,4,4,1,3,0,2
1375,0,2,0,0,0,6,0,2,2,5,0,0,0,0,0,2,0,2,0,0


In [72]:
y = df['label']
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
y.head(10)

597     1
700     1
1222    0
1145    1
602     0
1078    0
65      1
745     0
478     0
67      0
Name: label, dtype: int64

We will perform a 10-fold cross-validation to measure the performance of the classification model.

In [73]:
kfold = 10
X = X
y = encoded_y

random_state = 1

xval_accuracy = np.zeros(kfold)
xval_recall = np.zeros(kfold)
xval_f1 = np.zeros(kfold)
skf = KFold(n_splits = kfold, shuffle = True, random_state = random_state)
k = 0
epochs = 50
batch_size = 128

models = []
for train_index, xval_index in skf.split(X, y):
    X_train, X_xval = X.iloc[train_index], X.iloc[xval_index]
    y_train, y_xval = y[train_index], y[xval_index]
    
    model = Sequential()
    model.add(Dense(64, input_shape = (X_train.shape[1],))) 
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)
    models.append(model)
    
    y_pred = model.predict(X_xval).round().astype(int)

    xval_accuracy[k] = sklearn.metrics.accuracy_score(y_xval, y_pred)
    xval_recall[k] = sklearn.metrics.recall_score(y_xval, y_pred)
    xval_f1[k] = sklearn.metrics.f1_score(y_xval, y_pred)
    k+=1
    
print ('Average accuracy score', np.mean(xval_accuracy))
print ('Average recall score', np.mean(xval_recall))
print ('Average f1 score', np.mean(xval_f1))

39/39 [==============================] - 0s 2ms/step
Average accuracy score 0.6900983814662012
Average recall score 0.6393100195909442
Average f1 score 0.6697798854255439


In [74]:
xval_accuracy

array([0.68115942, 0.68115942, 0.71014493, 0.69565217, 0.69565217,
       0.64492754, 0.73913043, 0.7173913 , 0.67883212, 0.65693431])

In [75]:
xval_accuracy.argmax()

6

## Test accuracy

In [76]:
# Loading the test data
df_test = pd.read_csv('../../data/acp/test_data.csv')
print(df_test.shape)
df_test

(344, 2)


,sequences,label
0,FLPLLLSALPSFLCLVFKKC,0
1,DKLIGSCVWLAVNYTSNCNAECKRRGYKGGHCGSFLNVNCWCET,0
2,AVKDTYSCFIMRGKCRHECHDFEKPIGFCTKLNANCYM,0
3,GLPTCGETCFGGTCNTPGCTCDPWPVCTHN,1
4,ENCGRQAG,0
...,...,...
339,LLKKLLKWLKK,0
340,GLLLDTLKGAAKDIAGIALEKLKCKITGCKP,0
341,FWGALAKGALKLIPSLFSSFSKKD,1
342,GCLEFWWKCNPNDDKCCRPKLKCSKLFKLCNFSF,0


In [77]:
counts_test = [[Counter(i)[list_chars[j]] for j, c in enumerate(list_chars)] for i in list(df_test["sequences"])]
counts_test

[[1, 2, 0, 0, 3, 0, 0, 0, 2, 7, 0, 0, 2, 0, 0, 2, 0, 1, 0, 0],
 [2, 6, 1, 2, 1, 5, 1, 1, 3, 3, 0, 5, 0, 0, 2, 3, 2, 3, 2, 2],
 [2, 5, 2, 2, 3, 2, 2, 2, 4, 1, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2],
 [0, 6, 1, 1, 1, 5, 1, 0, 0, 1, 0, 2, 4, 0, 0, 0, 6, 1, 1, 0],
 [1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
 [2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 1, 0, 1, 5, 0, 0],
 [6, 0, 0, 0, 7, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [2, 1, 0, 0, 2, 4, 1, 2, 4, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
 [6, 0, 0, 1, 0, 5, 0, 1, 6, 5, 1, 1, 0, 0, 0, 1, 0, 2, 1, 0],
 [0, 0, 0, 0, 3, 3, 0, 1, 1, 5, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 0, 2, 4, 0, 1, 4, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 8, 5, 2, 0, 1, 3, 3, 5, 2, 2, 3, 2, 0, 3, 4, 2, 0, 0, 2],
 [2, 2, 2, 2, 3, 9, 1, 6, 8, 2, 1, 1, 0, 0, 1, 1, 1, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 3, 7, 1, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0],
 [2, 2, 2, 1, 0, 1, 0, 2, 4, 3, 0, 3, 1, 2, 3, 2, 1, 1, 0, 0],
 [2, 0, 2, 0, 1, 0, 1, 0, 3, 3, 1, 2, 1, 0, 3, 1, 0, 1,

In [78]:
len(counts_test)

344

In [79]:
df_counts_test = pd.DataFrame(data=counts_test, columns=list_chars)
df_counts_test["id"] = list(range(df_test.shape[0]))
df_counts_test

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,id
0,1,2,0,0,3,0,0,0,2,7,...,0,2,0,0,2,0,1,0,0,0
1,2,6,1,2,1,5,1,1,3,3,...,5,0,0,2,3,2,3,2,2,1
2,2,5,2,2,3,2,2,2,4,1,...,2,1,0,2,1,2,1,0,2,2
3,0,6,1,1,1,5,1,0,0,1,...,2,4,0,0,0,6,1,1,0,3
4,1,1,0,1,0,2,0,0,0,0,...,1,0,1,1,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,0,0,0,0,0,0,0,0,5,5,...,0,0,0,0,0,0,0,1,0,339
340,4,2,2,1,0,4,0,3,6,6,...,0,1,0,0,0,2,0,0,0,340
341,3,0,1,0,3,2,0,1,4,4,...,0,1,0,0,4,0,0,1,0,341
342,0,6,2,1,4,1,0,0,6,4,...,3,2,0,1,2,0,0,2,0,342


In [80]:
X_test = df_counts_test.set_index("id")
X_test

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
id,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,3,0,0,0,2,7,0,0,2,0,0,2,0,1,0,0
1,2,6,1,2,1,5,1,1,3,3,0,5,0,0,2,3,2,3,2,2
2,2,5,2,2,3,2,2,2,4,1,2,2,1,0,2,1,2,1,0,2
3,0,6,1,1,1,5,1,0,0,1,0,2,4,0,0,0,6,1,1,0
4,1,1,0,1,0,2,0,0,0,0,0,1,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,0,0,0,0,0,0,0,0,5,5,0,0,0,0,0,0,0,0,1,0
340,4,2,2,1,0,4,0,3,6,6,0,0,1,0,0,0,2,0,0,0
341,3,0,1,0,3,2,0,1,4,4,0,0,1,0,0,4,0,0,1,0


In [81]:
y_test = df_test['label']
y_test

0      0
1      0
2      0
3      1
4      0
      ..
339    0
340    0
341    1
342    0
343    1
Name: label, Length: 344, dtype: int64

In [83]:
# choosing the best model based on the xval
opt_model = models[xval_accuracy.argmax()]
y_pred_test = opt_model.predict(X_test).round().astype(int)
test_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred_test)
test_accuracy

11/11 [==============================] - 0s 2ms/step


0.6656976744186046

Given the simplicity of BoW embedding, accuracy of 66.6% is not bad!